In [8]:
import os
import json
import uuid

def json_read(path):
    with open(path, 'r') as fp:
        data = json.load(fp)
    return data

def json_dump(obj, path):
    with open(path, 'w') as fp:
        json.dump(obj, fp)

dataset_path = '/home/huanghongjun/FiCoVuL/FiCoVuL/data/datasets/DEPTH_RAW'
orig_path = '/home/huanghongjun/FiCoVuL/FiCoVuL/data/datasets/DEPTH_ORIG'
doxygen_json_path = '/home/huanghongjun/FiCoVuL/FiCoVuL/data/datasets/DEPTH_DOXYGEN_JSON'
result_path = '/home/huanghongjun/FiCoVuL/FiCoVuL/data/datasets/DEPTH_SINGULAR'
for split in ('train', 'valid', 'test'):
    dataset0 = json_read(os.path.join(dataset_path, f'{split}.json'))
    files = dataset0['graph_id_index']
    del dataset0
    files = list(map(lambda x: '-'.join(x.split('-')[:5])+'.json', files))
    def read_doxygen_json(fid):
        content = json_read(os.path.join(orig_path, fid))
        f = content['origin_path'].split('/')[-1]
        f = os.path.join(doxygen_json_path, f)
        content = json_read(f)
        functions = content['functions']
        mmap = {}
        for function in functions.keys():
            random_id = str(uuid.uuid4())
            func = functions[function]
            with open(os.path.join(result_path, split, f"{random_id}.c"), 'w') as f:
                code = func['code']
                f.write(code)
            annotation = {
                "origin_path": fid,
                "function_name": function,  # result 添加，replace
                "class": func['cwe_ids'],  # result 添加 extend
                "call_path": None,  # result 添加，extend
                "depth": 0,  # result 添加，int
                "label": int(func['label']) if func['label'] is not None else 0,  # result 添加，0/1/null（不用特殊考虑）
                "roi": func['lois'],  # result 添加，replace
            }
            json_dump(annotation, os.path.join(result_path, split, f"{random_id}.json"))
            mmap[function] = random_id
        temp = list(map(lambda x: list(map(mmap.get, x)), content['call_path']))
        json_dump(temp, os.path.join(result_path, split, 'annotations', f'{fid}.json'))
    list(map(read_doxygen_json, files))